David Rice, Amber Malpas, Armaan Goyal<br>
Architectures ML Project<br>
Manage Tables

In [33]:
import re
import numpy as np
import pandas as pd
import time

In [19]:
# comparison of Gini index calculations

def gini_ver1(ptemp):
    prats=[]
    for j in range(len(ptemp)-1):
        prats.append(ptemp[j+1]/ptemp[j])
    sums1=[]
    for j in prats:
        sums2=[]
        for k in prats:
            sums2.append(np.abs(j-k))
        sums1.append(np.sum(sums2))
    return (1/(2*len(prats)**2*np.mean(prats))*len(prats)/(len(prats)-1)*np.sum(sums1))

def gini_ver2(data, adj = True):
    diff = np.abs(np.subtract.outer(data, data))
    if adj:
        N = len(data)
        diff *= (N/(N-1))
    return np.mean(diff)/(2*np.mean(data))

# Monte Carlo test for Gini index from Deltas et al. (2003) - paper found at: https://www.jstor.org/stable/3211637
# generate 10,000 random uniform distributions each with size N = 50 and calculate mean Gini index
# Deltas et al. (2003) demonstrates mean value should be 0.333
# compare performance in terms of runtime and convergence to this mean Gini value

N = 10000
ver1 = np.empty(N)
ver2 = np.empty(N)
time1 = np.empty(N)
time2 = np.empty(N)

for i in range(N):
    arr = np.random.uniform(size = 50)
    start_time = time.time()
    ver1[i] = gini_ver1(arr)
    time1[i] = time.time() - start_time
    
    start_time = time.time()
    ver2[i] = gini_ver2(arr)
    time2[i] = time.time() - start_time

print(f"For Gini Version 1: Mean Gini is {np.mean(ver1): .3f}, Average Runtime is {np.mean(time1)} s")
print(f"For Gini Version 2: Mean Gini is {np.mean(ver2): .3f}, Average Runtime is {np.mean(time2)} s")

For Gini Version 1: Mean Gini is  0.707, Average Runtime is 0.002455119252204895 s
For Gini Version 2: Mean Gini is  0.335, Average Runtime is 4.152185916900635e-05 s


In [35]:
#Get all the data
inf=open('KOIData_Lissauer2023.csv','r')
lines=inf.readlines()

pltable=[] #List of all values for a planet
for line in lines[0:]:
    if line==lines[0]:
        colname=re.split(',',line) #Sheet column names
    else:
        div=re.split(',',line)
        pltable.append(div)

inf.close()

In [37]:
#Combine planets into system data
multis=[] #system & planet info for each system with 3+ planets
currentsys='x'
dumbplanets=[[]]
planetdata=['TTVperiod','radius','radius_em','radius_ep','nTT','SNR','MES','chisqwttv']
            #planet period, radius, radius minus error, radius plus error, number of transits with transit time measured, 
            #SNR, multiple event statistic, chi-squared with ttv (descriptions https://arxiv.org/pdf/2311.00238)
stardata=['kepmag','rhostar','mstar','logg','[M/H]']
            #Kepler Magniuted, Stellar density, Stellar Mass, Log gravity, metalicity (descriptiosn https://arxiv.org/pdf/2311.00238)
for i in range(len(pltable)):
    if pltable[i][colname.index('StatusFlag')][0]=='P' and 0<float(pltable[i][colname.index('TTVperiod')])<1000: #Planet candidate in most recent survey and less than 1000 days
        if pltable[i][colname.index('KOI')][0:-3]!=currentsys:
            if len(dumbplanets[0])>2:
                sort_ind=np.argsort(dumbplanets[0]) #sort by period
                multis.append([currentsys]+[float(pltable[i-1][colname.index(j)]) for j in stardata]+[np.array(k)[sort_ind] for k in dumbplanets])
                dumbplanets=[[float(pltable[i][colname.index(j)])] for j in planetdata]
            else:
                dumbplanets=[[float(pltable[i][colname.index(j)])] for j in planetdata]
            currentsys=pltable[i][colname.index('KOI')][0:-3]
        else:
            for j in range(len(dumbplanets)):
                dumbplanets[j].append(float(pltable[i][colname.index(planetdata[j])]))
                
newcolumns=['KOI']+stardata+planetdata #save column data

In [39]:
#Gap Complexity
def gapcomp(periods):
    cmax=[0.106,0.212,0.291,0.350,0.398,0.437,0.469,0.497]
    firstpar=[]
    secondpar=[]
    for j in range(len(periods)-1):
        p_i=np.log(periods[j+1]/periods[j])/np.log(periods[-1]/periods[0])
        firstpar.append(p_i*np.log(p_i))
        secondpar.append((p_i-(1/(len(periods)-1)))**2)
    return (-(1/cmax[len(periods)-3])*np.sum(firstpar)*np.sum(secondpar))

#Gini Index
def gini(ptemp):
    prats=[]
    for j in range(len(ptemp)-1):
        prats.append(ptemp[j+1]/ptemp[j])
    return gini_ver2(np.array(prats))

#Intra-system dispersion Weiss 2022 Equation 3
def qdis(ptemp):
    prats=[]
    for j in range(len(ptemp)-1):
        prats.append(ptemp[j+1]/ptemp[j])
    sums=[]
    for j in prats:
        sums.append(np.log10(j/np.mean(prats))**2)
    return np.sqrt(1/(len(ptemp)-1)*np.sum(sums))    

#median dp/p_i
def dpp(ptemp):
    dpptemp=[]
    for j in range(len(ptemp)-1):
        dpptemp.append((ptemp[j+1]-ptemp[j])/ptemp[j])
    return np.mean(dpptemp)
        
#median zeta
def zeta(ptemp):
    zetatemp=[]
    for j in range(len(ptemp)-1):
        prat=ptemp[j+1]/ptemp[j]
        zetatemp.append(np.abs(3*(1/(prat-1)-round(1/(prat-1)))))
    return np.mean(zetatemp)

In [41]:
#Add additional period data
newcols=newcolumns+['n','minper','maxper','meanper','perrange','lograngep','normstdp','meandpp','gapc','gini','qdisp','meanzeta']
for i in range(len(multis)):
    periods=multis[i][newcolumns.index('TTVperiod')]
    radiis=multis[i][newcolumns.index('radius')]
    multis[i].append(len(multis[i][newcolumns.index('TTVperiod')])) #number of planets
    multis[i].append(periods[0]) #Outer period
    multis[i].append(periods[-1]) #inner period
    multis[i].append(np.mean(periods)) #mean period (never seen this used but threw it in there)
    multis[i].append(periods[-1]-periods[0]) #period range
    multis[i].append(np.log10(periods[-1])-np.log10(periods[0])) #range in log period
    multis[i].append(np.std(periods)/np.mean(periods)) #normalzied standard deviation
    multis[i].append(dpp(periods)) #mean dp/p_inner
    multis[i].append(gapcomp(periods)) #gap complexity
    multis[i].append(gini(periods)) #Gini Index
    multis[i].append(qdis(periods)) #intra-system dispersion, q
    multis[i].append(zeta(periods)) #mean zeta

In [43]:
#intrasystem dispersion in radius
def qdisprad(rtemp):
    sums=[]
    for j in rtemp:
        sums.append(np.log10(j/np.mean(rtemp))**2)
    return np.sqrt(1/(len(rtemp))*np.sum(sums))  

#count planet types
def typecounter(rtemp):
    types=[0,0,0,0] #subearths, super-earths, sub-neptunes, neptune+
    for i in rtemp:
        if i<=1:
            types[0]=types[0]+1
        elif i<1.9:  #Using 1.9 Earth-radii for radius valley
            types[1]=types[1]+1 
        elif i<4: #4 for neptune radius approximately
            types[2]=types[2]+1
        else:
            types[3]=types[3]+1
    return types

In [45]:
##Add additional radius data
newcols=newcols+['minrad','maxrad','meanrad','radrange','stdr','normstdr','qdispr', 'gini_r', 'subearth','supearth','subnep','nepplus',]

for i in range(len(multis)):
    radiis=multis[i][newcolumns.index('radius')]
    multis[i].append(min(radiis)) #min radius
    multis[i].append(max(radiis))  #max radius
    multis[i].append(np.mean(radiis)) #mean radius
    multis[i].append(max(radiis)-min(radiis)) #range radius
    multis[i].append(np.std(radiis)) #radius standard deviation 
    multis[i].append(np.std(radiis)/np.mean(radiis)) #normalized standard deviation
    multis[i].append(qdisprad(radiis))
    multis[i].append(gini_ver2(radiis))
    multis[i]=multis[i]+typecounter(radiis)

In [49]:
#Write to file
outf=open('threeplanetsystemsinfo.csv','w')
for i in newcols:
    outf.write(i+',')
outf.write('\n')
for i in multis:
    for j in i:
        if isinstance(j, np.ndarray):
            outf.write(np.array2string(j, max_line_width=1000)+',')
        else:
            outf.write(str(j)+',')
    outf.write('\n')
outf.close()

In [37]:
solar=[88,225,365,687]
print(qdis(solar))
print(gapcomp(solar))
solarprat=[]
for i in range(3):
    solarprat.append(solar[i+1]/solar[i])
print(solarprat)
print(np.mean(solarprat))
print(qdis(solar)/np.mean(solarprat))
print(np.log10(qdis(solar)))

0.08264779296797711
0.12757698141193483
[2.5568181818181817, 1.6222222222222222, 1.8821917808219177]
2.0204107282874406
0.04090643145516843
-1.0827687393656946


In [19]:
#Kepler 286
k286=[1.796,3.468,5.914,29.22]
print(qdis(k286))
print(gapcomp(k286))
prat=[]
for i in range(3):
    prat.append(k286[i+1]/k286[i])
print(prat)

0.21280569434521746
0.4005081512395987
[1.930957683741648, 1.7053056516724336, 4.940818397024011]
0.5327872432742468
0.377227691921721


In [51]:
# read csv of system parameters into pandas dataframe where all data are floats or numpy arrays

def read_three_planet_csv(file, array_columns = ['TTVperiod', 'radius', 'radius_em', 'radius_ep', 'nTT', 'SNR', 'MES',
       'chisqwttv']):
    multi_df = pd.read_csv(file) # load file
    for col in array_columns: # iterate through columns that have array-like data (currently formatted as strings)
        str_list = multi_df[col]
        array_list = []
        for str in str_list:
            array = np.fromstring(str[1:-1], dtype=float, sep=' ') # convert string array from csv into numpy array
            array_list.append(array)
        multi_df[col] = array_list # replace all string arrays with numpy arrays
    return multi_df

multi_df = read_three_planet_csv("threeplanetsystemsinfo.csv")

In [53]:
# function to calculate all resonance-related statistic
# feed in period ratios, fractions corresponding to exact 1st and 2nd order resonances, and bounds for near-resonant classification (from Dai et al. 2024, https://arxiv.org/abs/2406.06885)

def delta_array(per_rats, first_order_fracs = [2, 3/2, 4/3, 5/4, 6/5], second_order_fracs = [3, 5/3, 7/5, 9/7], 
                first_order_bounds = [-.015, .03], second_order_bounds = [-.015, .015]):

    # generate array of delta values for 1st order resonances 
    first_deltas = np.divide.outer(per_rats, first_order_fracs) - 1 
    first_res = (first_deltas < first_order_bounds[1]) & (first_deltas > first_order_bounds[0])
    N_first = np.sum(first_res) # number of 1st order resonances

    # generate array of delta values for 2nd order resonances 
    second_deltas = np.divide.outer(per_rats, second_order_fracs) - 1
    second_res = (second_deltas < second_order_bounds[1]) & (second_deltas > second_order_bounds[0])
    N_second = np.sum(second_res) # number of 2nd order resonances

    # combine arrays for 1st and 2nd order resonances into single large array
    all_fracs = np.array(first_order_fracs + second_order_fracs)
    all_deltas = np.hstack([first_deltas, second_deltas]) 
    all_res = np.hstack([first_res, second_res])

    # locate minimum absolute delta values for each planet pair (i.e. how close is each pair to resonance)
    all_deltas_abs = np.abs(all_deltas) 
    inds = np.argmin(all_deltas_abs, axis = 1)
    inds_arr = np.eye(all_deltas.shape[1])[inds] 
    inds_arr = np.array(inds_arr, dtype = bool)
    min_deltas = all_deltas[inds_arr]

    # generate overall resonance summary (for each pair, 0 if non-resonant, else will provide exact MMR fraction of closest resonance
    all_fracs_arr = np.tile(all_fracs, (len(per_rats), 1))
    res_summary = np.max(all_res * all_fracs_arr, axis = 1)

    return N_first, N_second, min_deltas, res_summary

In [55]:
# apply resonance calculations to all three-planet systems and save as new csv file

N = len(multi_df)

N_first_list = []
N_second_list = []
min_deltas_list = []
min_abs_delta_list = []
med_abs_delta_list = []
res_summary_list = []

for i in range(N):
    sys = multi_df.loc[i]
    pers = sys['TTVperiod']
    per_rats = pers[1:]/pers[:-1]
    N_first, N_second, min_deltas, res_summary = delta_array(per_rats)
    N_first_list.append(N_first)
    N_second_list.append(N_second)
    min_abs_delta_list.append(np.min(np.abs(min_deltas)))
    med_abs_delta_list.append(np.median(np.abs(min_deltas)))
    min_deltas_list.append(min_deltas)
    res_summary_list.append(res_summary)

multi_df['N_first'] = N_first_list
multi_df['N_second'] = N_second_list
multi_df['deltas'] = min_deltas_list
multi_df['min_abs_delta'] = min_abs_delta_list
multi_df['med_abs_delta'] = med_abs_delta_list
multi_df['res_summary'] = res_summary_list

multi_df.to_csv('threeplanetsystemsinfo_with_res.csv', index=False, sep = ",") 

In [268]:
# open new csv file with resonance parameters

multi_df = read_three_planet_csv("threeplanetsystemsinfo_with_res.csv", array_columns = ['TTVperiod', 'radius', 'radius_em', 'radius_ep', 'nTT', 'SNR', 'MES',
       'chisqwttv', "deltas", "res_summary"])